Primero importa las librerias

In [1]:
# Load libraries
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Load sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings('ignore')

/home/nbuser/anaconda3_501/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Cargo mi dataset y reparto en caracteristicas normales y target

In [2]:
filename_reg = 'forestfires.csv'
data = pd.read_csv(filename_reg)
data1=data.drop(['month'], axis=1)
data2=data1.drop(['day'], axis=1)
#names=['X','Y','FFMC','DMC','DC','ISI','temp','RH','wind','rain','area']

#Dividamos la data 

array_reg = data2.values

X=array_reg[:,0:10]
Y=array_reg[:,10]

X_2=data2.drop("area",1)
Y_2=data2["area"]

Creo el conjunto de validación

In [3]:
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, 
                                               test_size=validation_size, random_state=seed)

Evaluo posibles modelos

In [4]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

# Spot Check Algorithms

models= []

models.append(( 'LiR',LinearRegression( )))
models.append(( 'Ridge',Ridge()))
models.append(( 'Lasso',Lasso()))
models.append(( 'ElN',ElasticNet()))
models.append(( 'KNN',KNeighborsRegressor()))
models.append(( 'CART',DecisionTreeRegressor(criterion='friedman_mse')))
models.append(( 'SVR',SVR(kernel='linear')))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print({name}, cv_results.mean(), ({cv_results.std()}))

{'LiR'} -4589.6847368683075 {8867.027717135163}
{'Ridge'} -4577.588724542246 {8855.3505172807}
{'Lasso'} -4566.373230096475 {8867.384485463146}
{'ElN'} -4563.3219644741575 {8866.265064002217}
{'KNN'} -5194.6673428218355 {8436.39089282725}
{'CART'} -9731.86849597706 {11273.223246453868}
{'SVR'} -4667.039413951965 {9184.562415583117}


Escojo al ElN ya que tiene el menor MSE. Uso el Grid Search para encontrar el mejor valor del hiperparametro 'alpha' para este modelo

In [5]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'


alpha = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])


param_grid=dict(alpha=alpha)

model = ElasticNet()
kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid= param_grid,scoring=scoring, cv=kfold)

grid_result = grid.fit(X_train, Y_train)


print({grid_result.best_score_.mean()}, {grid_result.best_estimator_.alpha})

{-4600.977505023031} {1.0}


Se observa que el valor optimo es 'alpha' = 1 y se obtiene con esto un MSE de 4600.977 

Esto me genera confusión puesto que antes se obtuvo para el ElN un MSE de 4563.142 en donde no se definió el hiperparametro 'alpha', sin embargo al no definirse este hiperparametro por default es 1 por lo tanto el MSE que obtengo con este GridSearch tambien deberia ser 4563.142

Adjunto el enlace en scikit-learn donde al inicio se menciona que por default para el ElasticNet Regression el 'alpha' es 1

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html